In [1]:
import pickle
import os
from perses.utils.smallmolecules import render_atom_mapping
import mdtraj as md
import numpy as np
import itertools

In [2]:
# Read in htfs

# ALA -> CYS
with open(f"/data/chodera/zhangi/perses_benchmark/neq/3/0/0_vacuum.pickle", 'rb') as f:
    htf_0 = pickle.load(f)

# THR -> SER
with open(f"/data/chodera/zhangi/perses_benchmark/neq/3/11/11_vacuum.pickle", 'rb') as f:
    htf_11 = pickle.load(f)

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
# Generate pdbs
traj = md.Trajectory(np.asarray(htf_0.old_positions(htf_0.hybrid_positions)), md.Topology.from_openmm(htf_0._topology_proposal.old_topology))
traj.save("ala.pdb")
traj = md.Trajectory(np.asarray(htf_0.new_positions(htf_0.hybrid_positions)), md.Topology.from_openmm(htf_0._topology_proposal.new_topology))
traj.save("cys.pdb")
traj = md.Trajectory(np.asarray(htf_11.new_positions(htf_11.hybrid_positions)), md.Topology.from_openmm(htf_11._topology_proposal.new_topology))
traj.save("thr.pdb")
traj = md.Trajectory(np.asarray(htf_11.new_positions(htf_11.hybrid_positions)), md.Topology.from_openmm(htf_11._topology_proposal.new_topology))
traj.save("ser.pdb")

In [2]:
# Copied from here: https://github.com/choderalab/perses/blob/ac9f4472c34ff2c6cd378fb33a9851968ec11bc6/perses/rjmc/topology_proposal.py#L1171
def generate_oemol_from_pdb_template(pdbfile):
    from perses.utils.openeye import createOEMolFromSDF
    current_oemol = createOEMolFromSDF(pdbfile, add_hydrogens = True)
    if not len(set([atom.GetName() for atom in current_oemol.GetAtoms()])) == len([atom.GetName() for atom in current_oemol.GetAtoms()]):
        raise Exception(f"the atoms in the oemol are not uniquely named.")

    #formatting all canonical atom names from pdb
    for atom in current_oemol.GetAtoms():
        name_with_spaces = atom.GetName()
        name_without_spaces = name_with_spaces.replace(" ", "")
        if name_without_spaces[0].isdigit():
            name_without_spaces = name_without_spaces[1:] + name_without_spaces[0]
        atom.SetName(name_without_spaces)
    return current_oemol


In [5]:
# Generate oemols
d_oemols = {}
for aa in ['ALA', 'CYS', 'THR', 'SER']:
    d_oemols[aa] = generate_oemol_from_pdb_template(f"{aa.lower()}.pdb")

INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


In [6]:
d_oemols

{'ALA': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x2b9fc2590450> >,
 'CYS': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x2b9fc25903c0> >,
 'THR': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x2b9fc25907b0> >,
 'SER': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x2b9fc2590810> >}

In [7]:
# Create list of tuples for every pair of amino acids
amino_acids = ['ALA', 'CYS', 'SER', 'THR']
pairs = list(itertools.permutations(amino_acids, r=2))


In [18]:
# Generate maps
directory = "/data/chodera/zhangi/perses_benchmark/neq/3/"

for i, pair in enumerate(pairs):
    print(pair)
    with open(os.path.join(directory, f"{i}/{i}_vacuum.pickle"), 'rb') as f:
        htf = pickle.load(f)
    traj = md.Trajectory(np.asarray(htf.old_positions(htf.hybrid_positions)), md.Topology.from_openmm(htf._topology_proposal.old_topology))
    traj.save("old.pdb")
    traj = md.Trajectory(np.asarray(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(htf._topology_proposal.new_topology))
    traj.save("new.pdb")
    
    old_oemol = generate_oemol_from_pdb_template("old.pdb")
    new_oemol = generate_oemol_from_pdb_template("new.pdb")
    print(os.path.join(directory, f"maps/{pair[0]}_{pair[1]}.pdf"))
#     render_atom_mapping(os.path.join(directory, f"maps/{pair[0]}_{pair[1]}.pdf"), old_oemol, new_oemol, htf._topology_proposal.new_to_old_atom_map)

#     render_atom_mapping(os.path.join(directory, f"maps/{pair[0]}_{pair[1]}.pdf"), d_oemols[pair[0]], d_oemols[pair[1]], htf._topology_proposal.new_to_old_atom_map)


INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


('ALA', 'CYS')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/ALA_CYS.pdf
('ALA', 'SER')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/ALA_SER.pdf
('ALA', 'THR')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/ALA_THR.pdf
('CYS', 'ALA')


INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


/data/chodera/zhangi/perses_benchmark/neq/3/maps/CYS_ALA.pdf
('CYS', 'SER')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/CYS_SER.pdf
('CYS', 'THR')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/CYS_THR.pdf
('SER', 'ALA')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/SER_ALA.pdf
('SER', 'CYS')


INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


/data/chodera/zhangi/perses_benchmark/neq/3/maps/SER_CYS.pdf
('SER', 'THR')


INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


/data/chodera/zhangi/perses_benchmark/neq/3/maps/SER_THR.pdf
('THR', 'ALA')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/THR_ALA.pdf
('THR', 'CYS')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/THR_CYS.pdf
('THR', 'SER')
/data/chodera/zhangi/perses_benchmark/neq/3/maps/THR_SER.pdf


## Get map for barnase:barstar

In [4]:
# Copied from here: https://github.com/choderalab/perses/blob/ac9f4472c34ff2c6cd378fb33a9851968ec11bc6/perses/rjmc/topology_proposal.py#L1171
def generate_oemol_from_pdb_template(pdbfile):
    from perses.utils.openeye import createOEMolFromSDF
    current_oemol = createOEMolFromSDF(pdbfile, add_hydrogens = True)
    if not len(set([atom.GetName() for atom in current_oemol.GetAtoms()])) == len([atom.GetName() for atom in current_oemol.GetAtoms()]):
        raise Exception(f"the atoms in the oemol are not uniquely named.")

    #formatting all canonical atom names from pdb
    for atom in current_oemol.GetAtoms():
        name_with_spaces = atom.GetName()
        name_without_spaces = name_with_spaces.replace(" ", "")
        if name_without_spaces[0].isdigit():
            name_without_spaces = name_without_spaces[1:] + name_without_spaces[0]
        atom.SetName(name_without_spaces)
    return current_oemol


In [6]:
barnase_oemol = generate_oemol_from_pdb_template("../input/mmc2_barnase.pdb")

INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


In [7]:
barstar_oemol = generate_oemol_from_pdb_template("../input/mmc2_barstar.pdb")

INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


AssertionError: the stereochemistry perception from 3D coordinates failed

In [8]:
# Generate maps
directory = "/data/chodera/zhangi/perses_benchmark/neq/6/"


with open(os.path.join(directory, f"{4}/{4}_apo.pickle"), 'rb') as f:
    htf = pickle.load(f)
traj = md.Trajectory(np.asarray(htf.old_positions(htf.hybrid_positions)), md.Topology.from_openmm(htf._topology_proposal.old_topology))
traj.save("old.pdb")
traj = md.Trajectory(np.asarray(htf.new_positions(htf.hybrid_positions)), md.Topology.from_openmm(htf._topology_proposal.new_topology))
traj.save("new.pdb")
    
old_oemol = generate_oemol_from_pdb_template("old.pdb")
new_oemol = generate_oemol_from_pdb_template("new.pdb")
print(os.path.join(directory, "maps/Y29F.pdf"))
#     render_atom_mapping(os.path.join(directory, f"maps/{pair[0]}_{pair[1]}.pdf"), old_oemol, new_oemol, htf._topology_proposal.new_to_old_atom_map)

#     render_atom_mapping(os.path.join(directory, f"maps/{pair[0]}_{pair[1]}.pdf"), d_oemols[pair[0]], d_oemols[pair[1]], htf._topology_proposal.new_to_old_atom_map)


INFO:numexpr.utils:Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:utils.openeye:molecule                       does not have unique atom names. Generating now...


AssertionError: the stereochemistry perception from 3D coordinates failed

In [13]:
unique_new = htf._topology_proposal.unique_new_atoms

In [12]:
unique_old = htf._topology_proposal.unique_old_atoms

In [14]:
for atom in htf._topology_proposal.new_topology.atoms():
    if atom.index in unique_new:
        print(atom)

<Atom 479 (HB2) of chain 0 residue 29 (PHE)>
<Atom 480 (HB3) of chain 0 residue 29 (PHE)>
<Atom 481 (HD1) of chain 0 residue 29 (PHE)>
<Atom 482 (HD2) of chain 0 residue 29 (PHE)>
<Atom 483 (HE1) of chain 0 residue 29 (PHE)>
<Atom 484 (HE2) of chain 0 residue 29 (PHE)>
<Atom 485 (HZ) of chain 0 residue 29 (PHE)>


In [15]:
for atom in htf._topology_proposal.old_topology.atoms():
    if atom.index in unique_old:
        print(atom)

<Atom 473 (CD2) of chain 0 residue 29 (TYR)>
<Atom 480 (HB2) of chain 0 residue 29 (TYR)>
<Atom 481 (HB3) of chain 0 residue 29 (TYR)>
<Atom 482 (HD1) of chain 0 residue 29 (TYR)>
<Atom 483 (HD2) of chain 0 residue 29 (TYR)>
<Atom 484 (HE1) of chain 0 residue 29 (TYR)>
<Atom 485 (HE2) of chain 0 residue 29 (TYR)>
<Atom 486 (HH) of chain 0 residue 29 (TYR)>


In [18]:
core_map = htf._topology_proposal.core_new_to_old_atom_map

In [21]:
new_atoms = list(htf._topology_proposal.new_topology.atoms())
old_atoms = list(htf._topology_proposal.old_topology.atoms())
for new, old in core_map.items():
    print(new_atoms[new], old_atoms[old])

<Atom 466 (N) of chain 0 residue 29 (PHE)> <Atom 466 (N) of chain 0 residue 29 (TYR)>
<Atom 467 (CA) of chain 0 residue 29 (PHE)> <Atom 467 (CA) of chain 0 residue 29 (TYR)>
<Atom 468 (C) of chain 0 residue 29 (PHE)> <Atom 468 (C) of chain 0 residue 29 (TYR)>
<Atom 469 (O) of chain 0 residue 29 (PHE)> <Atom 469 (O) of chain 0 residue 29 (TYR)>
<Atom 477 (H) of chain 0 residue 29 (PHE)> <Atom 478 (H) of chain 0 residue 29 (TYR)>
<Atom 478 (HA) of chain 0 residue 29 (PHE)> <Atom 479 (HA) of chain 0 residue 29 (TYR)>
<Atom 470 (CB) of chain 0 residue 29 (PHE)> <Atom 470 (CB) of chain 0 residue 29 (TYR)>
<Atom 471 (CG) of chain 0 residue 29 (PHE)> <Atom 471 (CG) of chain 0 residue 29 (TYR)>
<Atom 472 (CD1) of chain 0 residue 29 (PHE)> <Atom 472 (CD1) of chain 0 residue 29 (TYR)>
<Atom 473 (CD2) of chain 0 residue 29 (PHE)> <Atom 474 (CE1) of chain 0 residue 29 (TYR)>
<Atom 474 (CE1) of chain 0 residue 29 (PHE)> <Atom 475 (CE2) of chain 0 residue 29 (TYR)>
<Atom 475 (CE2) of chain 0 residue